# 3. Model Training 

## 3.1. Movement Data
Starting first with trying to predict User using statistical features of movement data, and testing different time intervels and it's effect on the accuracy of the results
### 3.1.1. Using 

In [1]:
# import numpy for math calculations
import numpy as np

# import pandas for data (csv) manipulation
import pandas as pd

# import gc to collect garbage
import gc

# import matplotlib for plotting
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('fivethirtyeight') 
%matplotlib inline

# import os for system operations
import os

# import sklearn for machine learning modelling and preprocessing
import sklearn

# importing Machine Learning Models
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

# importing from sklearn the evaluation metrics for classification
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

# importing from sklearn model selection 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, learning_curve,StratifiedShuffleSplit

from pathlib import Path

# import tqdm to show a smart progress meter
from tqdm.notebook import trange,tqdm

# import warnings to hide the unnessairy warniings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'catboost'

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def plot_importance(df):
    """
    Builds the dataset to plot the feature importance.
    
    """
    # Sort features according to importance
    df = df.sort_values(['Importance'],ascending=False).reset_index()
    
    # drop the old index to avoid confusion
    df = df.drop(['index'],axis=1)
    
    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 9))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:30]))), 
            df['Importance'].head(30), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:30]))))
    ax.set_yticklabels(df['Feature'].head(30))
    
    plt.xlabel("Normalized feature importance")
    plt.ylabel("Features")
    
    plt.show()
    return df

In [ ]:
classifiers = [['SVC',SVC(kernel="rbf", C=0.025, probability=True, random_state=SEED)],
               ['ExtraTreesClassifier', ExtraTreesClassifier(random_state=SEED)],
              ["LinearDiscriminantAnalysis", LinearDiscriminantAnalysis()],
              ['DecisionTreeClassifier',DecisionTreeClassifier(random_state=SEED)],
              ['KNeighborsClassifier', KNeighborsClassifier()],
              ['RandomForestClassifier', RandomForestClassifier(random_state=SEED)],
              ['MLPClassifier',MLPClassifier(random_state=SEED)],
              ['AdaBoostClassifier',AdaBoostClassifier(random_state=SEED)],
              ['NuSVC',NuSVC(probability=True,random_state=SEED)],
              ['GaussianNB', GaussianNB()],
              ['QuadraticDiscriminantAnalysis',QuadraticDiscriminantAnalysis()],
              ['LogisticRegression', LogisticRegression(random_state=SEED)],
              ['XGBClassifier', XGBClassifier(random_state=SEED)],
              ['BernoulliNB', BernoulliNB()],
              ['MultinomialNB', MultinomialNB()]]

In [ ]:
from sklearnex import patch_sklearn, config_context
patch_sklearn()

def evaluate_classifiers(classifiers):
    models = []
    Accuracy_set = pd.DataFrame(index=None, columns=['Model','Accuracy(Train)','Accuracy(Test)','F1(Train)','F1(Test)', 'Precision(Train)','Precision(Test)', 'Recall(Train)','Recall(Test)', 'Log_loss(Train)','Log_loss(Test)'])
    for i in tqdm(range(len(classifiers))):
        name = classifiers[i][0]
        model = classifiers[i][1]

        model.fit(X_train,y_train)

        y_train_predicted = model.predict(X_train)
        y_test_predicited = model.predict(X_test)

        accuracy_train = accuracy_score(y_train, y_train_predicted)
        accuracy_test = accuracy_score(y_test, y_test_predicited)

        f1_Score_train = f1_score(y_train, y_train_predicted,average='micro')
        f1_Score_test = f1_score(y_test, y_test_predicited,average='micro')

        precision_score_train = precision_score(y_train, y_train_predicted,average='micro')
        precision_score_test = precision_score(y_test, y_test_predicited,average='micro')

        recall_score_train = recall_score(y_train, y_train_predicted,average='micro')
        recall_score_test = recall_score(y_test, y_test_predicited,average='micro')

        log_loss_train = log_loss(y_train, model.predict_proba(X_train))
        log_loss_test = log_loss(y_test, model.predict_proba(X_test))

        # store the models
        models.append((name,accuracy_test,model))

        Accuracy_set = Accuracy_set.append(pd.Series({'Model':name, 'Accuracy(Train)':accuracy_train,'Accuracy(Test)':accuracy_test,'F1(Train)':f1_Score_train,'F1(Test)':f1_Score_test,'Precision(Train)':precision_score_train,'Precision(Test)':precision_score_test,'Recall(Train)':recall_score_train,'Recall(Test)':recall_score_test,'Log_loss(Train)':log_loss_train,'Log_loss(Test)':log_loss_test}),ignore_index=True )
    return Accuracy_set, models

In [ ]:
Accuracy_set, models = evaluate_classifiers(classifiers)
Accuracy_set.sort_values(by='Accuracy(Test)').style.background_gradient(cmap= plt.cm.Blues)

In [ ]:
plt.figure(figsize=(30,30))
g = sns.barplot(1,0, data = pd.DataFrame(models), palette="coolwarm",orient = "h")
g.set_xlabel("Mean Accuracy")
g = g.set_title("Cross validation scores")